## Worldwide Earthquake Events API - Gold Layer Processing

In [41]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
import reverse_geocoder as rg

StatementMeta(, 1d0ccab2-8c85-40fe-85c5-f47262fb4d1b, 45, Finished, Available, Finished)

In [42]:
df = spark.read.table("Earthquake_Lakehouse.earthquake_events_silver").filter(col('time') > start_date)

StatementMeta(, 1d0ccab2-8c85-40fe-85c5-f47262fb4d1b, 46, Finished, Available, Finished)

### Country Code Function

In [53]:
def get_country_code(lat, long):
    coordinates = (float(lat), float(long))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, 1d0ccab2-8c85-40fe-85c5-f47262fb4d1b, 57, Finished, Available, Finished)

### Registring UDFs

In [54]:
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, 1d0ccab2-8c85-40fe-85c5-f47262fb4d1b, 58, Finished, Available, Finished)

### Adding Country code and city attributes

In [55]:
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(, 1d0ccab2-8c85-40fe-85c5-f47262fb4d1b, 59, Finished, Available, Finished)

### Adding significance classification

In [57]:
df_with_location_sig_class = \
                            df_with_location.\
                            withColumn('sig_class', 
                                        when(col("sig") < 100, "Low").\
                                        when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                        otherwise("High")
                            )

StatementMeta(, 1d0ccab2-8c85-40fe-85c5-f47262fb4d1b, 61, Finished, Available, Finished)

### Appending the data to the gold table


In [58]:
df_with_location_sig_class.write.mode("overwrite").saveAsTable("earthquake_lakehouse.earthquake_events_gold")

StatementMeta(, 1d0ccab2-8c85-40fe-85c5-f47262fb4d1b, 62, Finished, Available, Finished)